# Money Diaries Data Reshape

In [1]:
import os
import re
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
pd.set_option('max_colwidth', 100)
pd.set_option('max_rows', 250)

In [3]:
dollar_pattern = r"\$\d+,?\d*(?:\.\d+)?"

## Build Wide `DataFrame`

In [4]:
def parse_total(s):
    total = re.findall(dollar_pattern, s)[-1]
    total = ''.join(total.split(','))
    return float(total.strip('$'))

In [5]:
def parse_salary(s):
    try:
        salary = re.findall(dollar_pattern, s)[0]
        salary = ''.join(salary.split(','))
        return float(salary.strip('$'))
    except IndexError:
        return 0

In [6]:
def add_totals_cols(df):
    for day in df.columns[:-2]:
        new_col = day + '_total'
        df[new_col] = df[day].apply(parse_total)
    
    totals = [c for c in df.columns if '_total' in c]
    df['weekly_total'] = df[totals].sum(axis=1)
    
    df['salary'] = df['title'].apply(parse_salary)
    return df

In [7]:
def cutoff_at_daily_total(s):
    # Found this regex pattern by carefully inspecting the text.
    # daily_total_pattern = "Total: " + dollar_pattern
    daily_total_pattern = "T[cha]*otal[ amount ]*[ sSpent]*: " + dollar_pattern
    day_seven_total = re.search(daily_total_pattern, s).group(0)
    cleaned = re.split(daily_total_pattern, s)[0].rstrip() + " " + day_seven_total
    return cleaned

In [8]:
def parse_location(s):
    trim_right = s.split("In ")[-1]
    trim_left = trim_right.split('On')[0]
    location = trim_left.strip().strip(',')
    return location

In [9]:
def add_location(df):
    df['location'] = df.title.apply(lambda x: parse_location(x))
    return df

In [10]:
def clean_trailing_text(df):
    df['day_seven'] = df.day_seven.apply(lambda x: cutoff_at_daily_total(x))
    return df

In [11]:
def hacky_replaces(s):
    '''
    There must be a better way...
    '''
    split = s.split('|')
    if ':' not in split[0]:
        split.pop(0)
        s = '|'.join(split)
    s = s.replace('||', '')
    s = s.replace('Monthly Expenses|', '')
    s = s.replace('Monthly expenses|', '')
    s = s.replace('Monthly Expenses:|', '')
    s = s.replace('Additional Expenses|', '')
    s = s.replace('Additional Expenses:|', '')
    s = s.replace('Annual Expenses|', '')
    s = s.replace('Yearly Expenses:|', '')
    s = s.replace('Yearly Expenses|', '')
    s = s.replace('Monthly Subscriptions/ Donations|', '')
    s = s.replace('S|', '')
    s = s.replace('hip:|$40Sub', 'hip:|$40|Sub')
    s = s.replace('ty:|$40Sub', 'ty:|$40|Sub')
    s = s.replace('Digit|', '')
    s = s.replace('Your Spending In Your State:|', '')
    s = s.replace('here|', '')
    s = s.replace('This week:|', '')
    s = s.replace('Shared My boyfriend and I have a joint account and each contribute $500/month to cover the following expenses (plus groceries, gas, and pet supplies):|', '')
    s = s.replace('Every month I withdraw $600 and divide it into physical envelopes for various expenses, including food ($200) and the following:|', '')
    s = s.replace('This week: A biomedical research analyst who makes $56,000 per year and spends it on bike accessories and a sleeping bag.|', '')
    s = s.replace('our guide to managing your money every day|', '')
    s = s.replace('All Other I put all expenses (below) on my credit card, which I pay in full every month. I have no credit card debt.|', '')
    s = s.replace('Monthly Revenue In Addition to Salary:|', '')
    s = s.replace('Monthly Loan Payments|Student Loans:|', 'Student Loans:|')
    s = s.replace('Loan Payments|Students Loans:|', 'Student Loans:|')
    s = s.replace('Split Expenses With My Boyfriend From Our Joint account|', '')
    s = s.replace('All Other Expenses|', '')
    s = s.replace(', for the inspiration.)|', '')
    s = s.replace("Editor's Note:|", '')
    s = s.replace('Hamilton|', '')
    s = s.replace('fe|si', 'fe si')
    s = s.replace('ly|th', 'ly th')
    s = s.replace('he|Su', 'he Su')
    s = s.replace('al.Sa', 'al|Sa')
    s = s.replace('ly|Ma', 'ly Ma')
    s = s.replace('ok.|My', 'ok. My')
    s = s.replace('is|good.', 'is good.')
    s = s.replace('ngs|get', 'ngs get')
    s = s.replace('Mortgage:$1,517', 'Mortgage:|$1,517')
    s = s.replace('fresh piecesOccupation:|', 'Occupation:|')
    s = s.replace('|:|', ':|')
    s = s.replace('taxes|,|etc.|', 'taxes, etc.')
    s = s.replace('All Other Utility bill: $40; Electricity bill: $40; Internet: $44; Car payment: $354; Car insurance: $93; Spotify: $10', 'All Other Utility bill:|$40|Electricity bill:|$40|Internet:|$44|Car payment:|$354|Car insurance:|$93|Spotify:|$10')
    s = s.replace('All Other Netflix: $10.73', 'Netflix:|$10.73')
    s = s.replace('through 401(k):|I contribute', 'through 401(k): I contribute')
    return s

In [12]:
def do_hacky_replaces(df):
    df['intro'] = df.intro.apply(lambda x: hacky_replaces(x))
    return df

In [13]:
filename = 'diaries.json'
wide_df = pd.read_json(filename)
wide_df = add_totals_cols(wide_df)
wide_df = add_location(wide_df)
wide_df = clean_trailing_text(wide_df)
wide_df = do_hacky_replaces(wide_df)

Wide `DataFrame` with cleaned, unshaped data.

## Build skinny `DataFrame` of Monthly Expenses

In [14]:
def row_to_expenses(txt):
    '''
    Return a small DataFrame of the monthly expenses listed for one diary.
    '''
    d = {}
    for i in range(0, len(txt.split('|')), 2):
        k = txt.split('|')[i].rstrip(':')
        v = txt.split('|')[i+1]
        d[k] = v
        
    df = pd.DataFrame.from_dict(d, orient='index')
    df = df.reset_index()
    df = df.rename_axis({'index': 'title', 0: 'value'},axis=1)
    
    return df

In [15]:
dfs = []
for row_ix in range(wide_df.shape[0]):
    small_expenses_df = row_to_expenses(wide_df.loc[row_ix, 'intro'])
    small_expenses_df['diary_index'] = row_ix
    dfs.append(small_expenses_df)
monthly_expense_df = pd.concat(dfs, axis=0)
monthly_expense_df.reset_index(inplace=True, drop=True)

In [16]:
monthly_expense_df.head()

title                              value  diary_index
0  Occupation  Digital Marketing Account Manager            0
1    Industry                        Hospitality            0
2         Age                                 26            0
3    Location                    Dublin, Ireland            0
4      Salary                            $37,100            0

Skinny `DataFrame` with each row representing a diarist's single monthly expense.

Mostly works but there is still some cleanup needed.  Might need to point-and-shoot e.g. in Excel

## Build skinny `DataFrame` of Daily Expenses

In [17]:
day_cols = ['day_one_total', 'day_two_total',
             'day_three_total', 'day_four_total',
             'day_five_total', 'day_six_total',
             'day_seven_total']

day_totals = wide_df[day_cols]

In [18]:
def row_to_day_totals(row_ix, day_totals):
    amounts = day_totals.loc[row_ix,:].values
    amounts = pd.DataFrame(amounts, columns=['day_total'])
    amounts['day_nm'] = np.arange(1,8)
    amounts['diary_index'] = row_ix
    return amounts

In [19]:
dfs = []
for row_ix in range(wide_df.shape[0]):
    df = row_to_day_totals(row_ix, day_totals)
    dfs.append(df)
daily_expense_df = pd.concat(dfs, axis=0)
daily_expense_df.reset_index(inplace=True, drop=True)

Skinny `DataFrame` with rows representing total daily expense in a diary-day.

## Build skinny `DataFrame` of diarists

In [20]:
diarist_df = wide_df[['salary', 'location', 'url']]
diarist_df.reset_index(level=0, inplace=True)

Skinny `DataFrame` with each row representing a diarist.

## Build skinny `DataFrame` of day text

In [21]:
day_text_cols = ['day_one', 'day_two',
             'day_three', 'day_four',
             'day_five', 'day_six',
             'day_seven']

day_text = wide_df[day_text_cols]

In [22]:
def row_to_day_text(row_ix, day_text):
    txt = day_text.loc[row_ix,:].values
    txt = pd.DataFrame(txt, columns=['day_rawtext'])
    txt['day_nm'] = np.arange(1,8)
    txt['diary_index'] = row_ix
    return txt

In [23]:
dfs = []
for row_ix in range(wide_df.shape[0]):
    df = row_to_day_text(row_ix, day_text)
    dfs.append(df)
daily_text_df = pd.concat(dfs, axis=0)
daily_text_df.reset_index(inplace=True, drop=True)

Skinny `DataFrame` holding raw text data.  Each row is a diary-day.

# Save to disk for manual cleaning

In [24]:
dataframes = {
    'monthly_expenses.csv': monthly_expense_df,
    'daily_expenses.csv': daily_expense_df,
    'diarists.csv': diarist_df,
    'daily_text.csv': daily_text_df,
}

In [25]:
directory = os.path.dirname(os.path.abspath(__name__))
if not os.path.exists(os.path.join(directory, 'data_reshape')):
    os.mkdir(os.path.join(directory, 'data_reshape'))
for basename, df in dataframes.items():
    savepath = os.path.join(directory, 'data_reshape', basename)
#     df.to_csv(savepath, index=False)

Uncomment the line above to enable the save.